In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from math import radians, cos, sin, asin, sqrt
import seaborn as sns
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from math import radians, cos, sin, asin, sqrt
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from random import sample
from pandas import DataFrame
from numpy import random
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from pandas import Series
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
from numpy import sqrt
from pandas import Series
import scipy
from scipy.stats import skew
from scipy.stats import kurtosis
import warnings
warnings.filterwarnings("ignore")
from numpy.random import seed
from numpy.random import randn
from scipy.stats import shapiro
import cudf
import cuml 

  ## What is the The Big 5 Personality Traits?
  
The Big Five personality traits is a suggested taxonomy, or grouping, for personality traits, developed from the 1980s onward in psychological trait theory.
Starting in the 1990s, the theory identified five factors by labels, for the US English speaking population, typically referred to as:


* openness to experience (inventive/curious vs. consistent/cautious)

* conscientiousness (efficient/organized vs. extravagant/careless)

* extraversion (outgoing/energetic vs. solitary/reserved)

* agreeableness (friendly/compassionate vs. critical/rational)

* neuroticism (sensitive/nervous vs. resilient/confident)


([Vikipedia](http://))




![](https://www.verywellmind.com/thmb/2VYS6NgHk4g0-oMD17MQ86wu_0w=/614x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/2795422-article-the-big-five-personality-dimensions-5a9083fb6edd650036603137.png)  
[Source](https://www.verywellmind.com/thmb/2VYS6NgHk4g0-oMD17MQ86wu_0w=/614x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/2795422-article-the-big-five-personality-dimensions-5a9083fb6edd650036603137.png)

 In that data have a 50 personality test question, partipiansts location, screen size and time information. I will analyze the data and then find the clusters. 

In [106]:
import cuml, pickle
from cuml.svm import SVR
print('RAPIDS version',cuml.__version__,'\n')

LOAD_SVR_FROM_PATH = '../input/svr-models-10-folds/'

df = pd.read_csv('../input/big-five-personality-test/IPIP-FFM-data-8Nov2018/data-final.csv', delimiter='\t')
df.head()

In [107]:
df.info(verbose=True)

In [108]:
#IPIP Scale contains reverse items, such as EXT2 "I dont talk a lot" in Extraversion scale. Giving high points of these question should be related with low level of extraversion.
#https://www.kaggle.com/code/gokceergun/cluster-analysis-five-factor-personality-model/notebook
df.EXT2 = 6 - df.EXT2.values
df.EXT4 = 6 - df.EXT4.values
df.EXT6 = 6 - df.EXT6.values
df.EXT8 = 6 - df.EXT8.values
df.EXT10 = 6 - df.EXT10.values
df.EST2 = 6 - df.EST2.values
df.EST4 = 6 - df.EST4.values
df.AGR1 = 6 - df.AGR1.values
df.AGR3 = 6 - df.AGR3.values
df.AGR5 = 6 - df.AGR5.values
df.AGR7 = 6 - df.AGR7.values
df.CSN2 = 6 - df.CSN2.values
df.CSN4 = 6 - df.CSN4.values
df.CSN6 = 6 - df.CSN6.values
df.CSN8 = 6 - df.CSN8.values
df.OPN2 = 6 - df.OPN2.values
df.OPN4 = 6 - df.OPN4.values
df.OPN6 = 6 - df.OPN6.values

##### I will be using that location data in the future, so I save like that.

In [109]:
country= pd.DataFrame(df["country"])
lat = pd.DataFrame(df["lat_appx_lots_of_err"])
long = pd.DataFrame(df["long_appx_lots_of_err"])

In [110]:
df.shape

In [111]:
df.isnull().sum()

In [112]:
percent= 100*(len(df.loc[:,df.isnull().sum(axis=0)>=1 ].index) / len(df.index))
print(round(percent,2))

In [113]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True,ascending=False)
missing_value_df

##### The missing value seems less because this dataset is really big. So I decided to drop all missing values.

In [114]:
df = df.dropna()
df.isnull().sum().any()

##### There can be multiple submissions from the same IP address. If the IP value is more than one, being recommended to delete it.

In [115]:
df.drop(df.index[df['IPC'] != 1], inplace = True)
df['IPC'].describe()

In [116]:
df.shape

##### I am removing non-numeric columns.

In [117]:
df.drop(['country','long_appx_lots_of_err','lat_appx_lots_of_err','dateload'],axis=1,inplace= True)

##### I've seen in some answers that there are 0 values but it shouldn't be like that. Answers should be between 1-5.

In [118]:
df =df[(df > 0).all(1)]

### Histogram 

In [119]:
def draw_histograms(dataframe, features, rows, cols):
    fig=plt.figure(figsize=(20,40))
    for i, feature in enumerate(features):
        ax=fig.add_subplot(rows,cols,i+1)
        df[feature].hist(bins=20,ax=ax,facecolor="b")
        ax.set_title(feature+" Distribution",color='DarkRed')
        ax.set_yscale('log')
    fig.tight_layout()  
    plt.show()
draw_histograms(df,df.columns,18,6)

##### Those histogram graphs not looks good I will do some skew analysis. 

In [120]:
pd.set_option('display.max_rows', None)
skew_list= pd.DataFrame(df.skew())
skew_list.reset_index(inplace = True)
skew_list
skew_list.rename(columns={"index": "Column", 0: "Skew"},inplace=True)

result = []
for i in skew_list["Skew"]:
    if -0.5 < i < 0.5 :
        result.append("symmetrical")
    if -1 < i < -0.5  :
        result.append("positive slightly_skewed")
    if 0.5 < i < 1  :
        result.append("negative slightly_skewed")
    if -1 > i:
        result.append("negative skewed")
    if  1 < i:
        result.append("positive skewed")
skew_list["Result"] = result

In [121]:
skew_label = list(skew_list["Result"].unique())
skew_list_values = list(skew_list["Result"].value_counts())

In [122]:
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = [8,4]
plt.title("Number of Columns")

ax.barh(skew_label, skew_list_values)
for i, value in enumerate(skew_list_values):
    ax.text(value + 3, i, str(value))
xmin, xmax = ax.get_xlim()
ax.set_xlim(xmin, 1.1*xmax)
plt.show()


In [123]:
pd.set_option('display.max_rows', None)
kurtosis_list = pd.DataFrame(df.kurtosis())
kurtosis_list.reset_index(inplace = True)
kurtosis_list.rename(columns={"index": "Column", 0: "kurtosis"},inplace=True)

result = []
for i in kurtosis_list["kurtosis"]:
    if i >3 :
        result.append("leptokurtic")
    if i <3:
        result.append("platykurtic")
    if i ==3:
        result.append("mesokurtic")
kurtosis_list["Result"] = result

In [124]:
kurtosis_list["Result"].value_counts()

In [125]:
kurtosis_list["Result"].value_counts().plot(kind='bar')
plt.xticks(rotation=30)

In [126]:
df.head()

### Handling Skewed Data

In [127]:
positive = skew_list[skew_list['Result'].str.contains("positive")]
negative = skew_list[skew_list['Result'].str.contains("negative")]

In [128]:
p = list(positive["Column"].values)
n = list(negative["Column"].values)

##### I using different methot for different skew type.

#### Box Cox For Posivite Skew

In [129]:
from scipy import stats 
for i in p:
    a, b = stats.boxcox(df[i])
    df[i] = a

#### Square Root For Negative Skew

In [130]:
for i in n:
    df[i] = np.sqrt((df[i]))

#### Transformation Result

In [131]:
pd.set_option('display.max_rows', None)
skew_list= pd.DataFrame(df.skew())
skew_list.reset_index(inplace = True)
skew_list
skew_list.rename(columns={"index": "Column", 0: "Skew"},inplace=True)

result = []
for i in skew_list["Skew"]:
    if -0.5 < i < 0.5 :
        result.append("symmetrical")
    if -1 < i < -0.5  :
        result.append("positive slightly_skewed")
    if 0.5 < i < 1  :
        result.append("negative slightly_skewed")
    if -1 > i:
        result.append("negative skewed")
    if  1 < i:
        result.append("positive skewed")
skew_list["Result"] = result

skew_label = list(skew_list["Result"].unique())
skew_list_values = list(skew_list["Result"].value_counts())

fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = [8,4]
plt.title("After The Data Transformation")

ax.barh(skew_label, skew_list_values)
for i, value in enumerate(skew_list_values):
    ax.text(value + 3, i, str(value))
xmin, xmax = ax.get_xlim()
ax.set_xlim(xmin, 1.1*xmax)
plt.show()



In [132]:
def draw_histograms(dataframe, features, rows, cols):
    fig=plt.figure(figsize=(20,40))
    for i, feature in enumerate(features):
        ax=fig.add_subplot(rows,cols,i+1)
        df[feature].hist(bins=20,ax=ax,facecolor="b")
        ax.set_title(feature+" Distribution",color='DarkRed')
        ax.set_yscale('log')
    fig.tight_layout()  
    plt.show()
draw_histograms(df,df.columns,18,6)

##### The distribution looks better.

### Shapiro-Wilk Test

In [133]:
seed(1)
stat, p = shapiro(df)
print('Statistics=%.3f, p=%.3f' % (stat, p))

alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

### Heatmap

##### I am checking that is there any correlation between answer point and answer time.

In [134]:
et=df.copy()
et.drop(et.columns[10:50], axis=1, inplace=True)
et.drop(et.columns[20:67], axis=1, inplace=True)
corrmat= et.corr().round(2)
plt.figure(figsize=(15,10))  
sns.heatmap(corrmat,annot=True,center=0)
plt.title("Extroversion Personality",fontsize =15)

In [135]:
es=df.copy()
es.drop(es.columns[0:10], axis=1, inplace=True)
es.drop(es.columns[10:50], axis=1, inplace=True)
es.drop(es.columns[20:57], axis=1, inplace=True)
corrmat= es.corr().round(2)
plt.figure(figsize=(15,10))  
sns.heatmap(corrmat,annot=True,center=0)
plt.title("Neuroticism Personality",fontsize =15)

In [136]:
ag=df.copy()
ag.drop(ag.columns[0:20], axis=1, inplace=True)
ag.drop(ag.columns[10:50], axis=1, inplace=True)
ag.drop(ag.columns[20:57], axis=1, inplace=True)
corrmat= ag.corr().round(2)
plt.figure(figsize=(15,10))  
sns.heatmap(corrmat,annot=True,center=0)
plt.title("Agreeable Personality",fontsize =15)

In [137]:
cs=df.copy()
cs.drop(cs.columns[0:30], axis=1, inplace=True)
cs.drop(cs.columns[10:50], axis=1, inplace=True)
cs.drop(cs.columns[20:57], axis=1, inplace=True)
corrmat= cs.corr().round(2)
plt.figure(figsize=(15,10))  
sns.heatmap(corrmat,annot=True,center=0)
plt.title("Conscientious Personality",fontsize =15)

In [138]:
op=df.copy()
op.drop(op.columns[0:40], axis=1, inplace=True)
op.drop(op.columns[10:50], axis=1, inplace=True)
op.drop(op.columns[20:57], axis=1, inplace=True)
corrmat= op.corr().round(2)
plt.figure(figsize=(15,10))  
sns.heatmap(corrmat,annot=True,center=0)
plt.title("Open Personality",fontsize =15)

I think that these correlation values are not too much. That's why I decided to drop the answer time columns.

In [139]:
df.drop(df.columns[50:100], axis=1, inplace=True)

In [140]:
df_sample = df[:60000]
df_sample.shape

In [141]:
pca = PCA(n_components=2)
pca_fit = pca.fit_transform(df)
df_pca = pd.DataFrame(data=pca_fit, columns=['PCA1', 'PCA2'])
plt.figure(figsize=(8,8))
sns.scatterplot(data=df_pca, x='PCA1', y='PCA2',  palette='Set2', alpha=0.8)
plt.title('Personality Clusters after PCA');

#### There is some outlier data on this graph. I am going to clean them with Dbscan algoritmh.

# DBSCAN

## Determining Epsilon

In [143]:
'''
from sklearn.neighbors import NearestNeighbors
neighbors = NearestNeighbors(n_neighbors=20)
neighbors_fit = neighbors.fit(df)
distances, indices = neighbors_fit.kneighbors(df)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)
plt.xlim([598000, 600000])
plt.ylim([0, 60])
'''

## Model

In [ ]:
DBS_clustering = DBSCAN(eps=300, min_samples=1200).fit(df)
df["label"] = DBS_clustering.labels_
df.head()

In [ ]:
df["label"].value_counts()

In [ ]:
pca = PCA(n_components=2)
pca_fit = pca.fit_transform(df)
df_pca = pd.DataFrame(data=pca_fit, columns=['PCA1', 'PCA2'])
df_pca['Clusters'] = DBS_clustering.labels_
plt.figure(figsize=(10,10))
sns.scatterplot(data=df_pca, x='PCA1', y='PCA2', hue='Clusters', palette='Set2', alpha=0.8)
plt.title('Personality Clusters after PCA');

In [ ]:
df.drop(df.index[df['label'] == -1], inplace = True)
df.shape

In [ ]:
df.drop(['label'],axis=1,inplace= True)

In [ ]:
df.head()

# K Means

In [ ]:
kmeans = KMeans()
visualizer = KElbowVisualizer(kmeans, k=(2,20))
visualizer.fit(df)
visualizer.poof()

In [ ]:
clusterer = KMeans(n_clusters=5)
preds = clusterer.fit_predict(df)
centers = clusterer.cluster_centers_

In [ ]:
pca = PCA(n_components=2)
pca_fit = pca.fit_transform(df)
df_pca = pd.DataFrame(data=pca_fit, columns=['PCA1', 'PCA2'])
df_pca['Clusters'] = preds
plt.figure(figsize=(8,8))
sns.scatterplot(data=df_pca, x='PCA1', y='PCA2', hue='Clusters', palette='Set2', alpha=0.8)
plt.title('Personality Clusters after PCA');

In [ ]:
score = silhouette_score(df, preds)
print("For n_clusters = {}, silhouette score is {})".format(5, score)) 

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for num_clusters in range_n_clusters:

    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(df)
    
    cluster_labels = kmeans.labels_
    
    silhouette_avg = silhouette_score(df, cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))

# GMM

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for num_clusters in range_n_clusters:
    gmm = GaussianMixture(n_components=num_clusters)
    gmm.fit(df)

    labels = gmm.predict(df)
    silhouette_avg = silhouette_score(df, labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))

In [ ]:
gmm = GaussianMixture(n_components=best)
gmm.fit(df)
labels = gmm.predict(df)

In [ ]:
pca = PCA(n_components=2)
pca_fit = pca.fit_transform(df)
df_pca = pd.DataFrame(data=pca_fit, columns=['PCA1', 'PCA2'])
df_pca['Clusters'] = labels
plt.figure(figsize=(8,8))
sns.scatterplot(data=df_pca, x='PCA1', y='PCA2', hue='Clusters', palette='Set2', alpha=0.8)
plt.title('Personality Clusters after PCA');

In [ ]:
score = silhouette_score(df, labels)
print("For n_clusters = {}, silhouette score is {})".format(5, score))

## DNN

##### I want to use neural network. It is will be like classification because when I building the model I will use k means labels.

In [ ]:
from tensorflow.keras import layers
import tensorflow as tf
def build_model():   
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, activation="relu", input_shape=(56,)))

    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Dense(64, activation="relu", kernel_regularizer = tf.keras.regularizers.l2()))
    
    model.add(layers.Dense(32, activation="relu"))
    
    model.add(layers.Dense(20, activation="softmax"))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy',])

    #model.summary()

    return model

In [ ]:
model = build_model()
save_best = tf.keras.callbacks.ModelCheckpoint("Model.h5", verbose=1, monitor='val_accuracy', save_best_only=True)

In [ ]:
preds = clusterer.fit_predict(df_sample)

In [ ]:
model.fit(df_sample, preds, validation_split=0.3, epochs=30, batch_size=128, shuffle=True, callbacks=[save_best])

In [ ]:
dnn_sample = df[60000:115828]
dnn_sample.shape

In [ ]:
df_sample.shape

In [ ]:
pred = model.predict(dnn_sample, verbose=1)
pred = np.argmax(pred, axis=1)
pd.DataFrame(pred)[0].value_counts()

In [ ]:
pred.shape

dnn_sample is new data for neural network model. 

In [ ]:
pca = PCA(n_components=2)
pca_fit = pca.fit_transform(df_sample)
df_pca = pd.DataFrame(data=pca_fit, columns=['PCA1', 'PCA2'])
df_pca['Clusters'] = pred
plt.figure(figsize=(8,8))
sns.scatterplot(data=df_pca, x='PCA1', y='PCA2', hue='Clusters', palette='Set2', alpha=0.8)
plt.title('Personality Clusters after PCA');

In [ ]:
score = silhouette_score(dnn_sample, pred)
print("For n_clusters = {}, silhouette score is {})".format(5, score)) 

# Results

I took these codes from this notebook. https://www.kaggle.com/code/akdagmelih/five-personality-clusters-k-means

In [ ]:
df.groupby('Clusters').mean()
col_list = list(df)
ext = col_list[0:10]
est = col_list[10:20]
agr = col_list[20:30]
csn = col_list[30:40]
opn = col_list[40:50]

data_sums = pd.DataFrame()
data_sums['extroversion'] = df[ext].sum(axis=1)/10
data_sums['neurotic'] = df[est].sum(axis=1)/10
data_sums['agreeable'] = df[agr].sum(axis=1)/10
data_sums['conscientious'] = df[csn].sum(axis=1)/10
data_sums['open'] = df[opn].sum(axis=1)/10
data_sums['clusters'] = preds
data_sums.groupby('clusters').mean()

In [ ]:
dataclusters = data_sums.groupby('clusters').mean()
plt.figure(figsize=(22,3))
for i in range(0, 5):
    plt.subplot(1,5,i+1)
    plt.bar(dataclusters.columns, dataclusters.iloc[:, i], color='green', alpha=0.2)
    plt.plot(dataclusters.columns, dataclusters.iloc[:, i], color='red')
    plt.title('Cluster ' + str(i))
    plt.xticks(rotation=45)
    plt.ylim(0,4);

In [ ]:
These graphs show us the personality traits of each clusters. 

#### Will be uptated...